In [1]:
%matplotlib inline

import cv2
import json
import numpy as np
import os
import pickle
import random
import sys
from datetime import datetime, timedelta, timezone

random.seed(1)
np.random.seed(1)

def show_and_wait(frame, title='tesst', wait_time=0):
    cv2.imshow(title, frame)
    key = cv2.waitKey(wait_time)
    return None if key < 0 else chr(key)

def read_and_wait(video, title='tesst', wait_time=0):
    result, frame = video.read()
    if result:
        return show_and_wait(frame, title, wait_time)

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = it.tee(iterable)
    next(b, None)
    return zip(a, b)

def yield_frames(video):
    while True:
        result, frame = video.read()
        if not result:
            break
        yield frame

In [ ]:
# Check a range of frames to ensure they're all Idle frames.
video = cv2.VideoCapture(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\champion\matafe_.mp4")
n = int(123 * video.get(cv2.CAP_PROP_FPS))
print(n)
print(video.set(cv2.CAP_PROP_POS_FRAMES, n))
for i in range(40):
    _, frame = video.read()
    show_and_wait(frame[:37, 137:265])
video.release()
input()
cv2.destroyAllWindows()

In [2]:
# Create an image representing an average Idle frame.
def sum_frames(file_path, start, count):
    try:
        video = cv2.VideoCapture(file_path)
        video.set(cv2.CAP_PROP_POS_FRAMES, start)
        return sum(f[:37, 137:265].astype('float32') for _, f in zip(range(count), yield_frames(video)))
    finally:
        video.release()
image = (sum_frames(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\champion\xednim1.mp4", 1409, 40) +
         sum_frames(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\idle\staycationyoutube1.mp4", 300, 40) +
         sum_frames(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\idle\vsnz1.mp4", 200, 40) +
         sum_frames(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\legends\mymisterfruit.mp4", 6060, 40)) / 160
image = image.astype('uint8')
show_and_wait(image)
cv2.destroyAllWindows()

In [3]:
# Create a mask to exclude the controller and label.
mask = np.ones_like(image) * 255
print(mask.shape)
mask[17:27, 33:99] = 0
top, left, right = 24, 1, 24
for i in range(0, mask.shape[0] - top):
    mask[top + i:top + i + 1, left + i:right + i] = 0
show_and_wait(mask)

# Create a function to apply the mask.
def apply_mask(image):
    return (mask & image).astype('float32')

# Apply the mask to the average image.
image = apply_mask(image)
show_and_wait(image.astype('uint8'))
cv2.destroyAllWindows()

(37, 128, 3)


In [ ]:
# Display the result.
video = cv2.VideoCapture(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\champion\xednim1.mp4")
video.set(cv2.CAP_PROP_POS_FRAMES, 1409)
for i in range(40):
    _, frame = video.read()
    cv2.imshow('Crop', frame[:37, 137:265])
    if show_and_wait(abs(image - apply_mask(frame[:37, 137:265])) / 255) == '?':
        print(i, sum(list(sum((image - apply_mask(frame[:37, 137:265])) ** 2).reshape(-1))))
cv2.destroyAllWindows()

In [4]:
# Report the largest squared difference from the average.
def get_largest_squared_difference(file_path, start, count, fn=max):
    try:
        video = cv2.VideoCapture(file_path)
        video.set(cv2.CAP_PROP_POS_FRAMES, start)
        l = [((image - apply_mask(f[:37, 137:265])) ** 2).reshape(-1) for _, f in zip(range(count), yield_frames(video))]
        return fn(map(sum, l)), fn(map(max, l))
    finally:
        video.release()

def print_largest_squared_difference(file_path, start, count, fn=max):
    print(os.path.basename(file_path), *get_largest_squared_difference(file_path, start, count, fn))

# Try it on known Idle frames.
print('Idle')
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\idle\staycationyoutube1.mp4", 300, 40)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\idle\vsnz1.mp4", 200, 40)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\legends\mymisterfruit.mp4", 6060, 40)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\legends\telshin.mp4", 12060, 40)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\legends\telshin.mp4", 16830, 400)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\unsupervised\telshin1.mp4", 210, 40)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\unsupervised\telshin1.mp4", 990, 40)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\champion\matafe_.mp4", 2538, 40)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\champion\xednim1.mp4", 1409, 400)

# Try it on known non-Idle frames.
print('non-Idle')
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\idle\staycationyoutube1.mp4", 0, 40, min)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\idle\vsnz1.mp4", 0, 40, min)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\legends\mymisterfruit.mp4", 0, 400, min)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\legends\telshin.mp4", 0, 400, min)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\legends\telshin.mp4", 18060, 400, min)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\legends\telshin.mp4", 19440, 400, min)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\unsupervised\telshin1.mp4", 1470, 400, min)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\champion\matafe_.mp4", 3717, 400, min)
print_largest_squared_difference(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\champion\xednim1.mp4", 0, 400, min)

Idle
staycationyoutube1.mp4 2209313.0 3249.0
vsnz1.mp4 372187.0 576.0
mymisterfruit.mp4 1101743.0 529.0
telshin.mp4 592576.0 3249.0
telshin.mp4 736480.0 3721.0
telshin1.mp4 728498.0 3721.0
telshin1.mp4 621553.0 2704.0
matafe_.mp4 1203361.0 2401.0
xednim1.mp4 2986419.0 4225.0
non-Idle
staycationyoutube1.mp4 40723296.0 42849.0
vsnz1.mp4 344141374.0 61009.0
mymisterfruit.mp4 22756873.0 20449.0
telshin.mp4 19068791.0 13225.0
telshin.mp4 28898611.0 13689.0
telshin.mp4 14309830.0 16129.0
telshin1.mp4 14149459.0 16384.0
matafe_.mp4 15517364.0 12996.0
xednim1.mp4 11986791.0 15876.0


In [5]:
# Save the average and mask images.
with open(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\idle\average.pickle", 'wb') as fout:
    pickle.dump(image, fout)
    pickle.dump(mask, fout)

# Save the sum and max parameters.
sum_upper_bound = 4e6
max_upper_bound = 5e3
with open(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\idle\average.json", 'wt') as fout:
    json.dump({'sum_upper_bound': sum_upper_bound, 'max_upper_bound': max_upper_bound},fout)

In [6]:
def is_idle(frame):
    frame = ((image - apply_mask(frame[:37, 137:265])) ** 2).reshape(-1)
    return sum(frame) < sum_upper_bound and max(frame) < max_upper_bound

def print_is_idle(file_path, start):
    try:
        video = cv2.VideoCapture(file_path)
        video.set(cv2.CAP_PROP_POS_FRAMES, start)
        print(is_idle(next(yield_frames(video))))
    finally:
        video.release()

# Try it on known Idle frames.
print('Idle')
print_is_idle(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\idle\staycationyoutube1.mp4", 300)
print_is_idle(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\idle\vsnz1.mp4", 200)
print_is_idle(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\legends\mymisterfruit.mp4", 6060)
print_is_idle(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\legends\telshin.mp4", 12060)
print_is_idle(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\legends\telshin.mp4", 16830)
print_is_idle(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\unsupervised\telshin1.mp4", 210)
print_is_idle(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\unsupervised\telshin1.mp4", 990)

# Try it on known non-Idle frames.
print('non-Idle')
print_is_idle(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\idle\staycationyoutube1.mp4", 0)
print_is_idle(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\idle\vsnz1.mp4", 0)
print_is_idle(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\legends\mymisterfruit.mp4", 0)
print_is_idle(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\legends\telshin.mp4", 0)
print_is_idle(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\legends\telshin.mp4", 18060)
print_is_idle(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\legends\telshin.mp4", 19440)
print_is_idle(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\unsupervised\telshin1.mp4", 1470)

Idle
True
True
True
True
True
True
True
non-Idle
False
False
False
False
False
False
False


In [ ]:
a = (np.random.random((2,3)) * 10).astype('uint8')
print(a)
b = (np.random.random((2,3)) * 10).astype('uint8')
print(b)
print(a.astype('float32') - b.astype('float32'))
print((a.astype('float32') - b.astype('float32')) ** 2)
print(sum(((a.astype('float32') - b.astype('float32')) ** 2).reshape(-1)))
print(max(((a.astype('float32') - b.astype('float32')) ** 2).reshape(-1)))

In [ ]:
[s for s in dir(cv2) if s.lower().startswith('canny')]

In [ ]:
video = cv2.VideoCapture(r"C:\Users\cidzerda\Documents\GitHub\strevr-data\idle\vsnz1.mp4")
print(video.get(cv2.CAP_PROP_FPS))
print(video.get(cv2.CAP_PROP_FRAME_COUNT))
start = 250
print(video.set(cv2.CAP_PROP_POS_FRAMES, start))
for i in it.count(300):
    _, frame = video.read()
    image = cv2.Canny(frame, 100, 100)
    print(image.shape)
    if show_and_wait(image, 'Crop').lower() == 'q':
        break
video.release()
cv2.destroyAllWindows()